---
license: apache-2.0
canonical_url: https://huggingface.co/docs/diffusers/training/basic_training
---

# 기본 훈련 예제

이 튜토리얼은 [`DiffusionPipeline`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)을 훈련시키는 간단한 예제를 보여줍니다. 이 예제는 [PyTorch](https://pytorch.org/)를 사용하여 작성되었지만, [Jax/Flax](https://github.com/huggingface/diffusers/tree/main/examples/basic_text_image_fine_tune)에서도 동일한 작업을 수행할 수 있습니다. 전체 훈련 스크립트는 [여기](https://github.com/huggingface/diffusers/blob/main/examples/unconditional_image_generation/train_unconditional.py)에서 찾을 수 있습니다.

훈련을 시작하기 전에 모든 필수 라이브러리가 설치되어 있는지 확인하십시오:

```bash
pip install diffusers accelerate transformers ftfy
```

이 튜토리얼에서는 [Smithsonian Butterfly Collection](https://huggingface.co/datasets/huggan/smithsonian_butterflies_subset) 데이터 세트에서 나비 이미지를 생성하도록 모델을 미세 조정합니다. 이 데이터 세트에는 1000개의 나비 이미지가 포함되어 있으며, 이는 모델이 새로운 것을 학습하기에 충분히 작습니다.

시작하려면 [Hugging Face Hub](https://huggingface.co/models)에서 사전 훈련된 모델을 로드합니다. 이 튜토리얼에서는 [`google/ddpm-cat-256`](https://huggingface.co/google/ddpm-cat-256) 체크포인트를 사용합니다. 이 체크포인트는 고양이 이미지에서 조건 없이 훈련되었습니다. 이 모델을 사용하여 나비를 생성하는 방법을 알아보겠습니다!

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 데이터 세트 구성

훈련을 위해 [🤗 Datasets](https://huggingface.co/docs/datasets/) 라이브러리를 사용하여 데이터 세트를 다운로드하고 전처리합니다. 다음과 같이 간단히 데이터 세트를 로드할 수 있습니다:

In [ ]:
from datasets import load_dataset

dataset_name = "huggan/smithsonian_butterflies_subset"
dataset = load_dataset(dataset_name, split="train")

다음으로, 이미지를 전처리합니다. 이미지 크기를 모델의 예상 입력과 일치하도록 조정하고, `ToTensor` 변환을 적용하여 이미지를 PyTorch 텐서로 변환합니다. 마지막으로, 이미지를 정규화합니다.

In [ ]:
from torchvision import transforms

image_size = 256
batch_size = 16

preprocess = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

데이터 세트에 이러한 변환을 적용하기 위해 `set_transform` 함수를 사용합니다:

In [ ]:
def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}


dataset.set_transform(transform)

그런 다음 데이터 세트를 `DataLoader`로 래핑하여 훈련 중에 데이터를 반복할 수 있도록 합니다.

In [ ]:
import torch

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

## 모델 구성

다음으로, [`UNet2DModel`](https://huggingface.co/docs/diffusers/main/en/api/models#diffusers.UNet2DModel)을 로드하고 구성합니다. 모델이 학습할 이미지의 크기와 일치하도록 `sample_size`를 설정해야 합니다. `layers_per_block`을 설정하여 UNet 아키텍처를 변경할 수도 있습니다. 이 매개변수는 각 다운샘플링 및 업샘플링 블록의 ResNet 블록 수를 정의합니다. 이 튜토리얼에서는 기본값을 유지합니다.

In [ ]:
from diffusers import UNet2DModel

model_name = "google/ddpm-cat-256"
model = UNet2DModel.from_pretrained(
    model_name, 
    use_safetensors=True,
    # num_train_timesteps=1000,
    # beta_start=0.0001,
    # beta_end=0.02,
    # beta_schedule="linear",
)

모델을 훈련시키는 동안 훈련 진행 상황을 모니터링하기 위해 모델의 샘플 출력을 주기적으로 저장하는 것이 좋습니다. `DDPMPipeline`은 이미지 생성을 위한 편리한 방법이며, 훈련 루프에서 사용됩니다.

In [ ]:
from diffusers import DDPMPipeline

image_pipe = DDPMPipeline(unet=model, scheduler=noise_scheduler)

## 스케줄러 구성

확산 모델은 노이즈 스케줄러를 사용하여 점진적으로 이미지에 노이즈를 추가하고 역으로 노이즈를 제거합니다. [`DDPMScheduler`](https://huggingface.co/docs/diffusers/main/en/api/schedulers#diffusers.DDPMScheduler)를 사용하여 훈련 중에 노이즈 제거 프로세스를 정의합니다.

In [ ]:
from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

## 훈련

이제 거의 모든 준비가 되었습니다! 마지막 단계는 옵티마이저를 설정하고 훈련 루프를 지정하는 것입니다. 이 튜토리얼에서는 AdamW 옵티마이저를 사용합니다.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-4)

이제 훈련 루프를 작성할 수 있습니다. 루프는 데이터를 반복하고, 각 단계에서 모델은 노이즈가 있는 이미지를 예측하려고 시도합니다. 그런 다음 손실 함수는 모델의 예측과 실제 값(이 예에서는 평균 제곱 오차) 간의 차이를 계산합니다. 그런 다음 손실을 사용하여 모델의 기울기를 업데이트합니다.

훈련 루프는 [🤗 Accelerate](https://huggingface.co/docs/accelerate)를 사용하여 분산 환경에서 훈련을 처리합니다. `Accelerator` 객체를 인스턴스화한 다음 훈련 객체를 `prepare` 함수로 보냅니다. 또한 Accelerate는 `gradient_accumulation_steps` 인수를 지원하여 효과적인 배치 크기를 늘려 그래디언트 누적을 수행합니다.

In [ ]:
from diffusers.optimization import get_cosine_schedule_with_warmup

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

이제 훈련 루프를 실행할 수 있습니다!

In [ ]:
from diffusers.utils import make_image_grid
import os

output_dir = "ddpm-butterflies-256"
num_epochs = 50

for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        clean_images = batch["images"].to(device)
        # Sample noise to add to the images
        noise = torch.randn(clean_images.shape).to(clean_images.device)
        bs = clean_images.shape[0]

        # Sample a random timestep for each image
        timesteps = torch.randint(
            0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device
        ).long()

        # Add noise to the clean images according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

        # Get the model prediction
        noise_pred = model(noisy_images, timesteps, return_dict=False)[0]

        # Calculate the loss
        loss = F.mse_loss(noise_pred, noise)
        loss.backward(loss)

        # Update the model parameters with the optimizer
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # After each epoch, log a few images from the pipeline to see how training is going
    if (epoch + 1) % 5 == 0:
        pipeline = DDPMPipeline(unet=model, scheduler=noise_scheduler) # TODO: make a pipeline from the model
        images = pipeline(
            batch_size = 4, 
            generator=torch.manual_seed(0),
        ).images
        # make_image_grid(images, rows=2, cols=2).save(f"{output_dir}/{epoch:04d}.png")
        print('Loss:', loss.item())

훈련이 완료되면 훈련된 모델을 [Hugging Face Hub](https://huggingface.co/models)에 업로드하여 모든 사람이 사용할 수 있도록 합니다. 모델을 업로드하려면 Hugging Face 계정에 로그인해야 합니다. 아직 계정이 없다면 [여기](https://huggingface.co/join)에서 계정을 만드십시오.

모델을 Hub에 푸시하기 위한 `push_to_hub` 함수를 만듭니다. 이 함수는 모델, 체크포인트 이름 및 커밋 메시지와 같은 몇 가지 매개변수를 받습니다.

In [ ]:
from huggingface_hub import HfApi, ModelCard, create_repo, get_full_repo_name

def save_model_card(repo_id: str, images=None, base_model=str, dataset_name=str, repo_folder=None):
    img_str = ""
    if images is not None:
        for i, ns_image in enumerate(images):
            # logger.info(f"Saving sample {i}.png")
            ns_image.save(os.path.join(repo_folder, f"sample_{i}.png"))
            img_str += f"![img{i}](./sample_{i}.png)"
    yaml = f"""
---    
base_model: {base_model}
instance_prompt: photograph of a [OBJECT] butterfly
tags:
- diffusers
- unconditional-image-generation
- diffusion-models-class
license: mit
inference: false
--- 
"""
    model_card = f"""
# DDPM - {repo_id}

This is a diffusion model trained on the {dataset_name} dataset.      

{img_str}
"""
    with open(os.path.join(repo_folder, "README.md"), "w") as f:
        f.write(yaml + model_card)


def upload_folder(repo_id, folder_path, path_in_repo):
    # 업로드할 파일 경로 목록 만들기
    files = [
        os.path.join(path, file)
        for path, _, files in os.walk(folder_path)
        for file in files
    ]
    # 각 파일을 repo에 업로드
    # logger.info(f"업로드 중인 파일 수: {len(files)}")
    for file_path in files:
        # repo 내 파일 경로 만들기
        destination_path = os.path.join(
            path_in_repo, os.path.relpath(file_path, folder_path)
        )
        # logger.info(f"'{file_path}' 업로드 중 -> '{destination_path}'")
        # 파일 업로드
        try:
            api.upload_file(
                path_or_fileobj=file_path,
                path_in_repo=destination_path,
                repo_id=repo_id,
            )
        except Exception as e:
            # logger.error(f"파일 업로드 중 오류 발생 {file_path}: {e}")
            pass

model_name = "ddpm-butterflies-10k_epochs"
hub_model_id = f"sasha/{model_name}"

pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)
pipeline.save_pretrained(output_dir)

repo_id = create_repo(hub_model_id, exist_ok=True, private=True).repo_id

with open(os.path.join(output_dir, "_config.json"), "r") as f:
    config = json.load(f)

save_model_card(
    repo_id,
    base_model=model_name,
    images=images,
    repo_folder=output_dir,
)
upload_folder(
    repo_id,
    output_dir,
    model_name,
)

이제 훈련된 모델을 Hub에 푸시했으므로 이제 누구나 [`DiffusionPipeline`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline) 클래스로 로드할 수 있습니다.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(hub_model_id, use_safetensors=True)
image = pipeline(num_inference_steps=250).images[0]
image

축하합니다! 이제 확산 모델을 훈련하고 Hub에 업로드하는 방법을 알게 되었습니다.